In [1]:
from azureml.core import Workspace
ws = Workspace(subscription_id="751bc669-6f29-4b8d-8a5c-ded54299723f",
               resource_group="play-estimator",
               workspace_name="play-estimator-ws")

In [2]:
import urllib.request
from azureml.core.model import Model

# Download model

# Register model
model = Model.register(ws, model_name="MSA_regmodel_3", model_path="./outputs/reg_model.pkl")

Registering model MSA_regmodel_3


In [58]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
# dummy_inference_config = InferenceConfig(
#     environment=env,
#     source_directory="./source_dir",
#     entry_script="echo_score.py",
# )
python_packages = ['joblib', 'pandas']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inf_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./echo_score.py')

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


In [74]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6799)

In [75]:
service = Model.deploy(
    ws,
    "myservice5",
    [model],
    inf_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_19e3c6a5a651c2b5f1598e216869e4a8
 ---> 6c8b0854ee7f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 05056d50d4c8
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6Ijc1MWJjNjY5LTZmMjktNGI4ZC04YTVjLWRlZDU0Mjk5NzIzZiIsInJlc291cmNlR3JvdXBOYW1lIjoicGxheS1lc3RpbWF0b3IiLCJhY2NvdW50TmFtZSI6InBsYXktZXN0aW1hdG9yLXdzIiwid29ya3NwYWNlSWQiOiI2MmMyMGNkYS00MzY3LTQ0NzctOTdjNS1iZDcwZDkxOWI5ZTkifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 02a2c6e20b02
 ---> 4279e3002b63
Step 4/5 : RUN mv '/var/azureml-app/tmpxrit9zm3.py' /var/azureml-app/main.py
 ---> Running in b3c6ff119f0f
 ---> 1e3ebab4d60b
Step 5/5 : CMD ["runsvdir","/va

In [76]:
import requests
import json
uri = service.scoring_uri

data = {
    'data': [
        10, 10, 3, 1, 2, 10, 3
    ]
}

headers = {"Content-Type": "application/json"}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

{'data': [5391.107033028812], 'message': 'Successfully predicted!'}


In [85]:
from azureml.core.webservice import AciWebservice, Webservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inf_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-07 17:51:15+00:00 Creating Container Registry if not exists.
2021-08-07 17:51:16+00:00 Registering the environment.
2021-08-07 17:51:18+00:00 Use the existing image.
2021-08-07 17:51:18+00:00 Generating deployment configuration.
2021-08-07 17:51:18+00:00 Submitting deployment to compute..
2021-08-07 17:51:28+00:00 Checking the status of deployment aciservice..
2021-08-07 17:54:02+00:00 Checking the status of inference endpoint aciservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [87]:
service = Webservice(workspace=ws, name="aciservice")
uri = service.scoring_uri

data = {
    'data': [
        10, 10, 3, 1, 2, 10, 3
    ]
}

headers = {"Content-Type": "application/json"}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

{'data': [5391.107033028812], 'message': 'Successfully predicted!'}
